In [2]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("resumeShikhar.pdf")
text_doc = loader.load()

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_documents = text_splitter.split_documents(text_doc)

In [4]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = OllamaEmbeddings(model="llama3")
db = Chroma.from_documents(splitted_documents, embeddings)


/var/folders/w8/58g_7p917x5gg9zz13xng8g80000gn/T/ipykernel_25834/180113476.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama3")


In [5]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3", base_url="http://localhost:11434")
llm

/var/folders/w8/58g_7p917x5gg9zz13xng8g80000gn/T/ipykernel_25834/3765493673.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3", base_url="http://localhost:11434")


Ollama(model='llama3')

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
I will tip you $1000 if the user finds the answer helpful.
<context>
{context}
</context>
Question: {input} """)

In [7]:
retriever = db.as_retriever()

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain
documents_chain = create_stuff_documents_chain(llm,prompt)

In [11]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,documents_chain)

In [13]:
response = retrieval_chain.invoke({"input": "what is shikhar's education?"})
response['answer']

"Based on the provided context, Shikhar Kanaujia's education details are:\n\n* Expected graduation year from Indian Institute of Information Technology (IIIT), Sonepat: 2027\n* Degree: B.Tech in Computer Science and Engineering\n* School: Delhi Public School, Kalyanpur\n* Class XII percentage: 92.6% (in 2023)\n* Class X percentage: 95.6% (in 2021)\n\nLet me know if you find this answer helpful!"